##Libreries

In [ ]:
pip install prophet


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import math
import scipy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import itertools
import random
import statsmodels.api as sm
# prophet by Facebook
from prophet import Prophet
# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")
sns.set()

##Carga de datos

In [ ]:
# Cargamos los datos
df = pd.read_csv('./BTCUSDT1h-18.8.17-31-7-22.csv',index_col=0, parse_dates=True)
df=df.drop(columns=['ignore'])
df=df.drop(columns=['open_time'])
df=df.drop(columns=['close_time'])

In [ ]:
df.head()

,open,high,low,close,volume,qav,num_trades,taker_base_vol,taker_quote_vol
Date,,,,,,,,,
2017-08-18 00:59:59.999,4285.08,4340.62,4228.76,4286.53,82.435474,353504.410409,268,70.703594,303212.284309
2017-08-18 01:59:59.999,4269.36,4269.36,4134.61,4243.59,75.684883,318327.529415,243,45.573568,192407.488931
2017-08-18 02:59:59.999,4251.95,4298.64,4234.00,4267.59,54.071337,230478.628087,219,46.316836,197398.076557
2017-08-18 03:59:59.999,4244.77,4308.70,4234.00,4292.39,64.001349,273495.006480,306,52.061572,222560.204200
2017-08-18 04:59:59.999,4292.39,4292.39,4234.43,4287.92,50.797515,216512.633035,267,45.826187,195365.989365


## Variables y adaptación de los datos

In [ ]:
#Variables
n_obs=math.floor(72)
freq='h'

#adaptación
df=df.asfreq(freq)
df=df.fillna(method='ffill')
df.isnull().sum()
#Check
nan_rows = df[df.isnull().any(1)]
nan_rows

,open,high,low,close,volume,qav,num_trades,taker_base_vol,taker_quote_vol
Date,,,,,,,,,


## Funciones para la representación gráfica

In [ ]:
warnings.simplefilter('ignore')

def show_graphp1(data,title):
    #data = pd.concat([df1, df2])
    data.reset_index(inplace=True, drop=True)
    #data=data.iloc[43200:]

    for col in data.columns:
        if col.lower().startswith('yhat'):
            data[col].plot(label=col,linestyle="dotted")
        else:
            data[col].plot(label=col)

    plt.rcParams["figure.figsize"] = (20, 10)
    plt.title(title)
    plt.legend()
    plt.show()
    print ()
    print('Evaluated model:', title)
    print('Mean absolute error:', mean_absolute_error(data['y'].values,data['yhat'].values))
    print('Root mean squared error:', np.sqrt(mean_squared_error(data['y'].values,data['yhat'].values)))

In [ ]:
warnings.simplefilter('ignore')

def show_graphp2(data,title):
    data.reset_index(inplace=True, drop=True)
    data=data.iloc[43200:]

    for col in data.columns:
        if col.lower().startswith('predicted_mean'):
            data[col].plot(label=col,linestyle="dotted")
        else:
            data[col].plot(label=col)

    plt.rcParams["figure.figsize"] = (20, 10)
    plt.title(title)
    plt.legend()
    plt.show()
   

##Test/train

In [ ]:
#train test todos los datos
df_train, df_test = df[0:-n_obs], df[-n_obs:]
#train test una variable
df1 = df.drop(['open', 'high','low', 'volume', 'qav','num_trades', 'taker_base_vol','taker_quote_vol'], axis=1)
df1_train, df1_test = df1[0:-n_obs], df1[-n_obs:]
#datasets para prophet
df1 = df1.reset_index()
df1.columns=['ds','y']
#dataset para obtener los resultados
df1_test = df1_test.reset_index()
df1_test.columns=['ds','y']
future=df1_test.drop(columns=['y'])

## Selección de fechas

In [ ]:
# create variables that will help us change the periods easily instead of typing them up every time
# make sure the start and end dates are business days, otherwise the code will result in an error
start_date = "2022-07-30 13:59:59.999"
end_date = "2022-07-31 00:59:59.999"

## Mape

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

##Mean absolute error y Root mean squared error

In [ ]:
    def errores(data,title):        
          print('Evaluated model:', title)
          print('Mean absolute error:', mean_absolute_error(data['y'].values,data['yhat'].values))
          print('Root mean squared error:', np.sqrt(mean_squared_error(data['y'].values,data['yhat'].values)))

## Preparación de los datos para el modelo PROPHET

In [ ]:
#Importación delibrerias del modelo
from prophet import Prophet

#Preparación del dataset para la estructura de PROPHET (una variable)
dfp = df.drop(['open', 'high','low', 'volume','qav','num_trades', 'taker_base_vol','taker_quote_vol'], axis=1)
dfp = dfp.reset_index()
dfp = dfp.rename(columns={ 'close':'y'})
dfp = dfp.rename(columns={ 'Date':'ds'})
dfp

,ds,y
0,2017-08-18 00:59:59.999,4286.53
1,2017-08-18 01:59:59.999,4243.59
2,2017-08-18 02:59:59.999,4267.59
3,2017-08-18 03:59:59.999,4292.39
4,2017-08-18 04:59:59.999,4287.92
...,...,...
43388,2022-07-30 20:59:59.999,23963.03
43389,2022-07-30 21:59:59.999,23860.51
43390,2022-07-30 22:59:59.999,23700.98
43391,2022-07-30 23:59:59.999,23643.51


##HyperParameter Tuning using ParameterGrid


In [ ]:
# from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#               'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'seasonality_prior_scale': [0.1,0.3,0.4,0.5,1,2,3,4,5,6,7,8,9,10],
#               'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200],
#               'changepoint_range':[0.81,0.82,0.83,0.84,0.85,0.86,0.87,0.88,0.89, 0.90,0.91,0.92,0.93,0.94,0.95],
#               }
# grid = ParameterGrid(params_grid)
# cnt = 0
# for p in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)

In [ ]:
from sklearn.model_selection import ParameterGrid
params_grid = {'seasonality_mode':('multiplicative','additive'),
              'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
              #'seasonality_prior_scale': [0.1,0.3,0.4,0.5,1,2,3,4,5,6,7,8,9,10],
              'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
              'n_changepoints' : [100,150,200]}
grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 150


##Prophet Model Tuning


In [ ]:
strt= start_date
end=end_date
model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
for p in grid:
    test = pd.DataFrame()
    print(p)
    random.seed(0)
    train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                         holidays_prior_scale = p['holidays_prior_scale'],
                         n_changepoints = p['n_changepoints'],
                         seasonality_mode = p['seasonality_mode'],
                         weekly_seasonality=True,
                         daily_seasonality = True,
                         yearly_seasonality = True,
                         interval_width=0.95)
    

    train_model.fit(dfp)
    train_forecast = train_model.make_future_dataframe(periods=72, freq='h',include_history = False)
    train_forecast = train_model.predict(train_forecast)
    test=train_forecast[['ds','yhat']]
    #Actual = df[(df['ds']>strt) & (df['ds']<=end)] df1_test
    #MAPE = mean_absolute_percentage_error(Actual['y'],abs(test['yhat']))
    MAPE = mean_absolute_percentage_error(df1_test['y'].values,test['yhat'].values)
    print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)
    model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)

{'changepoint_prior_scale': 0.1, 'holidays_prior_scale': 0.1, 'n_changepoints': 100, 'seasonality_mode': 'multiplicative'}


DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/g0_st_c4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/9yxnkhgx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=83280', 'data', 'file=/tmp/tmpdae5lj1n/g0_st_c4.json', 'init=/tmp/tmpdae5lj1n/9yxnkhgx.json', 'output', 'file=/tmp/tmp07_whzp5/prophet_model-20220919085142.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:51:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:00:09 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Mean Absolute Percentage Error(MAPE)------------------------------------ 1.1668039173316804
{'changepoint_prior_scale': 0.1, 'holidays_prior_scale': 0.1, 'n_changepoints': 100, 'seasonality_mode': 'additive'}


DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/64g6tngl.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/b5fl3zrb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=94487', 'data', 'file=/tmp/tmpdae5lj1n/64g6tngl.json', 'init=/tmp/tmpdae5lj1n/b5fl3zrb.json', 'output', 'file=/tmp/tmpvftck_yv/prophet_model-20220919090013.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:00:13 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:08:22 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Mean Absolute Percentage Error(MAPE)------------------------------------ 2.240090040155475
{'changepoint_prior_scale': 0.1, 'holidays_prior_scale': 0.1, 'n_changepoints': 150, 'seasonality_mode': 'multiplicative'}


...

DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/k96mlty3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/5nzcd63r.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=19224', 'data', 'file=/tmp/tmpdae5lj1n/k96mlty3.json', 'init=/tmp/tmpdae5lj1n/5nzcd63r.json', 'output', 'file=/tmp/tmpks7k8fke/prophet_model-20220919195404.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:54:04 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
20:05:45 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Mean Absolute Percentage Error(MAPE)------------------------------------ 1.407641626977726
{'changepoint_prior_scale': 0.2, 'holidays_prior_scale': 0.5, 'n_changepoints': 150, 'seasonality_mode': 'additive'}


DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/dt5fvvnq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/jr9iobjt.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=75879', 'data', 'file=/tmp/tmpdae5lj1n/dt5fvvnq.json', 'init=/tmp/tmpdae5lj1n/jr9iobjt.json', 'output', 'file=/tmp/tmpqumduesq/prophet_model-20220919200549.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
20:05:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
20:17:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Mean Absolute Percentage Error(MAPE)------------------------------------ 2.117960059906241
{'changepoint_prior_scale': 0.2, 'holidays_prior_scale': 0.5, 'n_changepoints': 200, 'seasonality_mode': 'multiplicative'}


DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/xu7k4fy9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdae5lj1n/fyezr4mj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=49074', 'data', 'file=/tmp/tmpdae5lj1n/xu7k4fy9.json', 'init=/tmp/tmpdae5lj1n/fyezr4mj.json', 'output', 'file=/tmp/tmp0i7xj10l/prophet_model-20220919201710.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
20:17:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


In [ ]:
parameters = model_parameters.sort_values(by=['MAPE'])
parameters = parameters.reset_index(drop=True)
parameters.head()

In [ ]:
parameters['Parameters'][0]

In [ ]:
## Modelo optmizado

In [ ]:
# Setup and train model with holidays
final_model = Prophet(holidays=holiday,
                      changepoint_prior_scale= 0.5,
                      holidays_prior_scale = 0.1,
                      n_changepoints = 200,
                      seasonality_mode = 'multiplicative',
                      weekly_seasonality=True,
                      daily_seasonality = True,
                      yearly_seasonality = True,
                      interval_width=0.95)
final_model.add_country_holidays(country_name='US')
final_model.fit(dfp)

In [ ]:
future = final_model.make_future_dataframe(periods=122, freq='D')
forecast = final_model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)